### Training RL Policies using L5Kit Closed-Loop Environment

This notebook describes how to train RL policies for self-driving using our gym-compatible closed-loop environment.

We will be using [Proximal Policy Optimization (PPO)](https://arxiv.org/abs/1707.06347) algorithm as our reinforcement learning algorithm, as it not only demonstrates remarkable performance but it is also empirically easy to tune.

The PPO implementation in this notebook is based on [Stable Baselines3](https://github.com/DLR-RM/stable-baselines3) framework, a popular framework for training RL policies. Note that our environment is also compatible with [RLlib](https://docs.ray.io/en/latest/rllib.html), another popular frameworks for the same.

In [ ]:
import os
os.environ["L5KIT_DATA_FOLDER"] = '/workspace/datasets'

In [ ]:
import gym

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import get_linear_fn
from stable_baselines3.common.vec_env import SubprocVecEnv

from l5kit.configs import load_config_data
from l5kit.environment.feature_extractor import CustomFeatureExtractor
from l5kit.environment.callbacks import L5KitEvalCallback
from l5kit.environment.envs.l5_env import SimulationConfigGym

from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet, CLEMetricSet
from prettytable import PrettyTable
import datetime
import pytz
import numpy as np

In [ ]:
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable
# get environment config
env_config_path = '/workspace/source/src/configs/gym_config.yaml'
cfg = load_config_data(env_config_path)

### Define Training and Evaluation Environments

**Training**: We will be training the PPO policy on episodes of length 32 time-steps. We will have 4 sub-processes (training environments) that will help to parallelize and speeden up episode rollouts. The *SimConfig* dataclass will define the parameters of the episode rollout: like length of episode rollout, whether to use log-replayed agents or simulated agents etc.

**Evaluation**: We will evaluate the performance of the PPO policy on the *entire* scene (~248 time-steps).

In [ ]:
# Train on episodes of length 32 time steps
train_eps_length = 32
train_envs = 4

# Evaluate on entire scene (~248 time steps)
eval_eps_length = None
eval_envs = 1

# make train env
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
env = make_vec_env("L5-CLE-v0", env_kwargs=env_kwargs, n_envs=train_envs,
                   vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

# make eval env
validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
eval_env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, \
                   'return_info': True, 'train': False, 'sim_cfg': validation_sim_cfg}
eval_env = make_vec_env("L5-CLE-v0", env_kwargs=eval_env_kwargs, n_envs=eval_envs,
                        vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

### Define backbone feature extractor

The backbone feature extractor is shared between the policy and the value networks. The feature extractor *simple_gn* is composed of two convolutional networks followed by a fully connected layer, with ReLU activation. The feature extractor output is passed to both the policy and value networks composed of two fully connected layers with tanh activation (SB3 default).

We perform **group normalization** after every convolutional layer. Empirically, we found that group normalization performs far superior to batch normalization. This can be attributed to the fact that activation statistics change quickly in on-policy algorithms (PPO is on-policy) while batch-norm learnable parameters can be slow to update causing training issues.

In [ ]:
# A simple 2 Layer CNN architecture with group normalization
model_arch = 'simple_gn'
features_dim = 128

# Custom Feature Extractor backbone
policy_kwargs = {
    "features_extractor_class": CustomFeatureExtractor,
    "features_extractor_kwargs": {"features_dim": features_dim, "model_arch": model_arch},
    "normalize_images": False
}

### Clipping Schedule

We linearly decrease the value of the clipping parameter $\epsilon$ as the PPO training progress as it shows improved training stability

In [ ]:
# Clipping schedule of PPO epsilon parameter
start_val = 0.1
end_val = 0.01
training_progress_ratio = 1.0
clip_schedule = get_linear_fn(start_val, end_val, training_progress_ratio)

### Hyperparameters for PPO. 

For detailed description, refer https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/ppo/ppo.html#PPO

In [ ]:
lr = 3e-4
num_rollout_steps = 256
gamma = 0.8
gae_lambda = 0.9
seed = 42
batch_size = 64
import datetime
tensorboard_log = '/content/drive/MyDrive/Colab Notebooks/l5kit/tb_logs/'  + str(datetime.date.today()) + '/'

In [ ]:
config = {
    'lr' : 3e-4,
    'num_rollout_steps' :  256, # 2048
    'gamma' : 0.8,
    'gae_lambda' : 0.9,
    'seed' : 42,
    'batch_size' : 64, # 512
    }

In [ ]:
import wandb
from wandb.integration.sb3 import WandbCallback
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'sac_policy_training.ipynb'
hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")

experiment_name = f"SAC_sb3_{date}"

run = wandb.init(
    project="l5kit2",
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=False,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
    reinit = True,
    group = 'SAC',
    config = config,
    name = experiment_name
)

### Define the PPO Policy.

SB3 provides an easy interface to the define the PPO policy. Note: We do need to tweak appropriate hyperparameters and the custom policy backbone has been defined above.


In [ ]:

# model = SAC('MultiInputPolicy', env, policy_kwargs=policy_kwargs, verbose=1,  batch_size=batch_size,  learning_rate= lr,  gamma=gamma, tensorboard_log=tensorboard_log, buffer_size=10000,
#             learning_starts=batch_size) 
model = SAC("MultiInputPolicy", env, batch_size=batch_size, learning_rate= lr,
                    gamma=gamma,
                    buffer_size=int(1e5),
                    tensorboard_log=f"runs/{run.id}", 
                    train_freq= (1, "step"),
                    gradient_steps=1, # gradient 1 time per 1 episode
                    target_update_interval=200, # each ...(200) steps, we update target_net (load weight from policy_net)
                    learning_starts=batch_size,
                    policy_kwargs=policy_kwargs,
                    verbose=1,
                    seed=42,
                    use_sde=True,
                    sde_sample_freq=-1,
                    use_sde_at_warmup=True,
                    tau=.95, 
                    # create_eval_env=True,
                    )

### Defining Callbacks

We can additionally define callbacks to save model checkpoints and evaluate models during training.

In [ ]:
callback_list = []

# Save Model Periodically
os.environ['WANDB_NOTEBOOK_NAME'] = 'sac_policy_training.ipynb'
hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")
save_freq = 10000
save_path = '/DATA/l5kit/logs/'+ date
output = 'SAC'
checkpoint_callback = CheckpointCallback(save_freq=(save_freq // train_envs), save_path=save_path, \
                                         name_prefix=output)
callback_list.append(checkpoint_callback)

# Eval Model Periodically
eval_freq = 10000
n_eval_episodes = 1
val_eval_callback = L5KitEvalCallback(eval_env, eval_freq=(eval_freq // train_envs), \
                                      n_eval_episodes=n_eval_episodes, n_eval_envs=eval_envs)
callback_list.append(val_eval_callback)
callback_list.append(WandbCallback(
        # gradient_save_freq=100,
        # model_save_path=f"models/{run.id}",
        # verbose=2,
        # model_save_freq=10,
    ))

### Train

In [ ]:
n_steps = 6e6

model.learn(n_steps, callback=callback_list)

run.finish()

**Voila!** We have a trained PPO policy! Train for larger number of steps for better accuracy. Typical RL algorithms require training atleast 1M steps for good convergence. You can visualize the quantitiative evaluation using tensorboard.

In [ ]:
model = SAC.load('/workspace/datasets/logs/06-01-2023_15-15-53/SAC_6000000_steps.zip', env = eval_env , custom_objects = {
      # "learning_rate": 0.0,
      # "lr_schedule": lambda _: 0.0,
      # "clip_range": lambda _: 0.0,
  })
n_steps = 1000000
model.learn(n_steps, callback=callback_list, reset_num_timesteps=False)

### Visualize the episode from the environment

We can easily visualize the outputs obtained by rolling out episodes in the L5Kit using the Bokeh visualizer.

In [ ]:
model = SAC.load('/workspace/datasets/logs/06-01-2023_15-15-53/SAC_6000000_steps.zip', 
                 env = eval_env, 
                 custom_objects = {
                      "learning_rate": 0.0,
                      "lr_schedule": lambda _: 0.0,
                      "clip_range": lambda _: 0.0,}
                )

In [ ]:
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
rollout_env = gym.make("L5-CLE-v0", env_config_path=env_config_path, sim_cfg=rollout_sim_cfg, \
                       use_kinematic=True, train=False, return_info=True)

from src.simulation.unrollGym import unroll, visualize_outputs
sim_outs = unroll(model, rollout_env,100)

In [ ]:
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
rollout_env = gym.make("L5-CLE-v0", env_config_path=env_config_path, sim_cfg=rollout_sim_cfg, \
                       use_kinematic=True, train=False, return_info=True)

from src.simulation.unrollGym import unroll
sim_outs = unroll(model, rollout_env)

## Calculate the performance metrics from the episode outputs

We can also calculate the various quantitative metrics on the rolled out episode output. 

In [ ]:
from src.validate.validator import compute_ade_fde, quantify_outputs
ades, fdes = compute_ade_fde(sim_outs)
quantify_outputs(sim_outs)

In [ ]:
print(round(np.mean(ades),2), round((np.std(ades)),2) ,round((np.max(ades) - np.min(ades))/2,2))
print(round(np.mean(fdes),2), round((np.std(fdes)),2) ,round((np.max(fdes) - np.min(fdes))/2,2))
sorted_ades_idx = sorted(range(len(ades)), key=lambda i: ades[i], reverse=True)
sorted_fdes_idx = sorted(range(len(fdes)), key=lambda i: fdes[i], reverse=True)
print(f'Top worst scence based on ade:{sorted_ades_idx[-10:]}')
print(f'Top best scence based on ade:{sorted_ades_idx[:-10:-1]}')
print(f'Top worst scence based on fde:{sorted_fdes_idx[-10:]}')
print(f'Top best scence based on fde:{sorted_fdes_idx[:-10:-1]}')
print(f'mean, min/max, std: {round(np.mean(ades),2), round((np.std(ades)),2) ,round((np.max(ades) - np.min(ades))/2,2)}')
print(f'mean, min/max, std: {round(np.mean(fdes),2), round((np.std(fdes)),2) ,round((np.max(fdes) - np.min(fdes))/2,2)}')

### Best ade

In [ ]:
# might change with different rasterizer
sim_outs_selected =  [sim_outs[i] for i in sorted_ades_idx[-5:-1]]
output_notebook()
visualize_outputs(sim_outs_selected)

### Worst ade

In [ ]:
sim_outs_selected =  [sim_outs[i] for i in sorted_ades_idx[:5]]
output_notebook()
visualize_outputs(sim_outs_selected)### Worst ade

## Use Validator

In [ ]:
from src.validate.validator import CLEValidator
import matplotlib.pyplot as plt
import numpy as np

agg = CLEValidator(sim_outs)
fields = ["metric", "value"]
table = PrettyTable(field_names=fields)

values = []
names = []

for metric_name in agg:
    table.add_row([metric_name, agg[metric_name].item()])
    values.append(agg[metric_name].item())
    names.append(metric_name)

print(table)

plt.bar(np.arange(len(names)), values)
plt.xticks(np.arange(len(names)), names, rotation=60, ha='right')
plt.show()